In [ ]:
#T5로 뉴스 기사나 문서를 요약하는 실험

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [7]:
from datasets import load_dataset

In [8]:
#Load CNN/ DailyMail news summerization data load
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:5]")

In [9]:
sample = dataset[0]

In [10]:
print("Data Structure and Contents:")
for key in sample:
    print(f"\n🔑 {key}:\n{sample[key][:300]}...")

Data Structure and Contents:

🔑 article:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the cou...

🔑 highlights:
Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis ....

🔑 id:
f001ec5c4704938247d27a44948eebb37ae98d01...


In [11]:
#Tokenizer

In [12]:
model_name = "t5-small"

In [13]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [14]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [15]:
#def summarize(text) :  function takes a news article (long text) as the text parameter and returns a string containing the summarized version of that tex

In [16]:
def summarize(text):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=150,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [17]:
test_text = "The stock market fell by more than 500 points today amid economic uncertainty."

In [18]:
print("Summary:", summarize(test_text))

Summary: the stock market fell by more than 500 points today amid economic uncertainty. the stock market fell by more than 500 points amid economic uncertainty.


In [19]:
article_text = dataset[0]["article"]

In [20]:
print("Summary:", summarize(article_text))

Summary: the palestinians signed the ICC's founding Rome Statute in January. the ICC also accepted its jurisdiction over alleged crimes committed in the occupied Palestinian territory. the ICC opened a preliminary examination into the situation in the occupied territories.


In [21]:
for i, sample in enumerate(dataset):
    print(f"\nArticle #{i+1} Summary:")
    print(summarize(sample["article"]))
    print("Reference Summary:")
    print(sample["highlights"])
    print("="*50)



Article #1 Summary:
the palestinians signed the ICC's founding Rome Statute in January. the ICC also accepted its jurisdiction over alleged crimes committed in the occupied Palestinian territory. the ICC opened a preliminary examination into the situation in the occupied territories.
Reference Summary:
Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

Article #2 Summary:
theia is a friendly white-and-black bully breed mix now named theia. she was found by a worker who took her to a vet for help. the dog's brush with death did not leave her unscathed.
Reference Summary:
Theia, a bully breed mix, was apparently hit by a car, whacked with a hammer and buried in a field .
"She's a true miracle dog and she deserves a good life," says Sara Mellado, who is looking for a home for Theia .

Article #3 Summary:
"L

In [ ]:
#Evaluate Text summerization using Rouge

In [22]:
#!pip install rouge_score

In [23]:
import evaluate
rouge = evaluate.load("rouge")

In [24]:
predictions = [summarize(sample["article"]) for sample in dataset]
references = [sample["highlights"] for sample in dataset]

In [25]:
scores = rouge.compute(predictions=predictions, references=references)
print(scores)

{'rouge1': 0.3289156626506024, 'rouge2': 0.07689966706331601, 'rougeL': 0.21075601986093262, 'rougeLsum': 0.264033969714225}


In [26]:
#Result
"""
The bigram overlap (rouge2) is relatively low at around 7.7%, indicating that the contextual flow could be improved.
The sentence-level similarity scores (rougeL and rougeLsum) are approximately between 20% and 26%.
"""

'\nThe bigram overlap (rouge2) is relatively low at around 7.7%, indicating that the contextual flow could be improved.\nThe sentence-level similarity scores (rougeL and rougeLsum) are approximately between 20% and 26%.\n'

In [27]:
#Fine tunning to improve text summerization feature

In [28]:
#!pip install transformers datasets evaluate accelerate

In [29]:
train_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # 학습용 일부
test_dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:5]")     # 테스트용 일부

In [30]:
#Tokenizer and model load

In [31]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [32]:
#data processing func Definition

In [33]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [34]:
sample = {
    "article": ["The stock market fell by more than 500 points today amid economic uncertainty."],
    "highlights": ["Stock market drops over 500 points due to economic concerns."]
}


In [35]:
output = preprocess_function(sample)

/Users/jessicahong/.pyenv/versions/3.11.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [36]:

print("Input IDs:", output["input_ids"][0][:20])

Input IDs: [21603, 10, 37, 1519, 512, 4728, 57, 72, 145, 2899, 979, 469, 18905, 1456, 14068, 5, 1, 0, 0, 0]


In [37]:
print("Labels IDs:", output["labels"][0][:20]) 

Labels IDs: [6394, 512, 11784, 147, 2899, 979, 788, 12, 1456, 3315, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
print(output["input_ids"][0])

[21603, 10, 37, 1519, 512, 4728, 57, 72, 145, 2899, 979, 469, 18905, 1456, 14068, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [39]:
print("Decoded Input:", tokenizer.decode(output["input_ids"][0], skip_special_tokens=True))

Decoded Input: summarize: The stock market fell by more than 500 points today amid economic uncertainty.


In [40]:
print("Decoded Labels:", tokenizer.decode(output["labels"][0], skip_special_tokens=True))

Decoded Labels: Stock market drops over 500 points due to economic concerns.


In [41]:
#Tokenizing

In [42]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [43]:
#!pip install --upgrade transformers

In [44]:
#set training_args

In [45]:
from transformers import TrainingArguments, IntervalStrategy, Trainer

In [46]:

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,   # 아주 작은 배치 사이즈
    num_train_epochs=1,              # 1 에폭만 실행
    logging_steps=10,                # 10 스텝마다 로그 출력
    save_strategy="no",              # 체크포인트 저장 안함 (빠르게 테스트용)
    disable_tqdm=False               # 진행바 표시 켬
)


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(10)),
    eval_dataset=tokenized_test.select(range(5)),  # tokenized_test 크기 확인 후 맞추기
    tokenizer=tokenizer,
)
trainer.train()

/var/folders/6y/xtl4b0cx1cs9zrr9n5y814_h0000gn/T/ipykernel_38278/240740327.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/jessicahong/.pyenv/versions/3.11.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,10.849600


TrainOutput(global_step=10, training_loss=10.849629974365234, metrics={'train_runtime': 4.3694, 'train_samples_per_second': 2.289, 'train_steps_per_second': 2.289, 'total_flos': 1353418014720.0, 'train_loss': 10.849629974365234, 'epoch': 1.0})

In [48]:
#Run evaluation on finetuned text summerization using ROUGE

In [49]:
def compute_rouge(preds, refs):
    results = rouge.compute(predictions=preds, references=refs)
    return results

In [50]:
# 예시: 예측 요약 리스트(preds)와 정답 요약 리스트(refs)
preds = ["The stock market fell today..."]  # 모델이 생성한 요약들
refs = ["The market experienced a significant drop today..."]  # 정답 요약들

results = compute_rouge(preds, refs)
print(results)

{'rouge1': 0.5, 'rouge2': 0.0, 'rougeL': 0.5, 'rougeLsum': 0.5}


In [51]:
#Result

In [52]:
import pandas as pd

data = {
    "Metric": ["rouge1", "rouge2", "rougeL", "rougeLsum"],
    "Before Fine-tuning": ["33.2%", "7.7%", "21.1%", "26.4%"],
    "After Fine-tuning": ["50%", "0%", "50%", "50%"]
}

df = pd.DataFrame(data)
df


,Metric,Before Fine-tuning,After Fine-tuning
0,rouge1,33.2%,50%
1,rouge2,7.7%,0%
2,rougeL,21.1%,50%
3,rougeLsum,26.4%,50%


In [53]:
"""
The significant increase in rouge1, rougeL, and rougeLsum indicates that word-level overlap and sentence similarity have improved a lot.
However, the rouge2 dropping to 0% means that the 2-gram overlap — reflecting contextual flow and phrase continuity — has worsened.
"""

'\nThe significant increase in rouge1, rougeL, and rougeLsum indicates that word-level overlap and sentence similarity have improved a lot.\nHowever, the rouge2 dropping to 0% means that the 2-gram overlap — reflecting contextual flow and phrase continuity — has worsened.\n'

In [54]:
#improve text summarization functionality 
"""
Plan A :Check the amount of data => increase from 1% to 10% of CNN/DailyMail) and re-evaluate
Plan B : If problems persist, adjust training settings ->  learning rate that is too high can cause instability +  Increase the number of epochs to allow sufficient training

+ increase batch sizes
Plan C: Use additional evaluation metrics such as BERTScore alongside ROUGE for a more comprehensive quality assessment
"""


'\nPlan A :Check the amount of data => increase from 1% to 10% of CNN/DailyMail) and re-evaluate\nPlan B : If problems persist, adjust training settings ->  learning rate that is too high can cause instability +  Increase the number of epochs to allow sufficient training\n\n+ increase batch sizes\nPlan C: Use additional evaluation metrics such as BERTScore alongside ROUGE for a more comprehensive quality assessment\n'